#### Taking the infinite limit

We have the pre-limit expression:

$$
=\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\cdot
\frac{K!}{K_0!K^{K^+}}
\cdot
\left(
\frac
  {N!}
  {\prod_{j=1}^{N}(j + \alpha/K)}
  \right)^{K}
\cdot
\prod_{k=1}^{K^+}
\left(
\frac
  {(N-m_k)!\, \prod_{j=1}^{m_k} (j + \alpha/K)}
  {N!}
  \right)
$$

We've already seen that:

$$
\lim_{K \to \infty}
\frac{K!}{K_0!K^{K^+}}
= 1
$$
(from the section on latent classes earlier)

We've also seen that:

$$
\lim_{K \to \infty}
\left(
\frac
  {N!}
  {\prod_{j=1}^{N}(j + \alpha/K)}
  \right)^{K}
= \exp \left(
    -\alpha\sum_{j=1}^N \frac{1}{j}
  \right)
$$

So, we're left with taking the limit of:

$$
\lim_{K \to \infty}
\prod_{k=1}^{K^+}
\left(
\frac
  {(N-m_k)!\, \prod_{j=1}^{m_k} (j + \alpha/K)}
  {N!}
  \right)
$$

My earlier intuition would be to just remove the expression in $\alpha/K$, giving:

$$
\prod_{k=1}^{K^+}
\left(
\frac
  {(N-m_k)!\, m_k!}
  {N!}
  \right)
$$
